# Вебинар 1. Введение, примеры задач, бизнес- и ML-метрики 

### Домашнее задание 
### ML-mетрики качества

In [67]:
import pandas as pd
import numpy as np

### 1. Hit rate

In [68]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

def hit_rate_at_k(recommended_list, bought_list, k=5):
#   сделать в домашней работе
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k] # Добавлен срез
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

In [69]:
recommended_list = [143,156, 1134, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [70]:
print(hit_rate(recommended_list, bought_list))
print(hit_rate_at_k(recommended_list, bought_list))

1
1


### 2. Precision

In [72]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [73]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


def precision_at_k(recommended_list, bought_list, k=5):
#     сделать дома
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k] # Добавлен срез
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def money_precision_at_k_(recommended_list, bought_list, prices_recommended, k=5):
    
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [76]:
print(precision(recommended_list, bought_list))
print(precision_at_k(recommended_list, bought_list, k=3))

0.2
0.3333333333333333


### 3. Recall

In [77]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
#    сделать дома
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k] # Добавлен срез
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    #    сделать дома
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(recommend_list, bought_list)
    
    recall = np.dot(flags, prices_recommended).sum() / prices_bought.sum() # Относильно money_precision_at_k_ изменен знаменатель
    
    return recall

In [78]:
print(recall(recommended_list, bought_list))
print(recall_at_k(recommended_list, bought_list))

0.5
0.5


# Метрики ранжирования

## AP@k

In [79]:
recommended_list = [221,21,3234,1,234,234,234,666] #id товаров
bought_list = [1,221,3,4,5,6,7,8,9]

In [80]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant

In [81]:
ap_k(recommended_list, bought_list, k=5)

0.75

### MAP@k

In [82]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]] # юзер 3

In [83]:
def map_k(recommended_list, bought_list, k=5):
    # сделать дома
    sum_apk = 0
    for user_index in range(len(bought_list)):
        sum_apk += ap_k(recommended_list[user_index], bought_list[user_index], k)
    result = sum_apk/len(bought_list)
    return result

In [84]:
map_k(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333

### Normalized discounted cumulative gain ( NDCG@k)

In [85]:
# по желанию
def dcg(list_of_indexes, lenght):
    result = 0
    for i in list_of_indexes:
        if i == 0:
            result += 1/1
        elif i == 1:
            result += 1/2
        else:
            result += 1/np.log(i+1)
    return result/lenght
    

def ndcg_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommend_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    ndcg = dcg(relevant_indexes, k)/dcg([index for index in range(k)], k)

    return ndcg

In [86]:
ndcg_at_k(recommended_list, bought_list, k=5)

0.458668649780196

### Mean Reciprocal Rank ( MRR@k )

In [87]:
def reciprocal_rank(recommended_list, bought_list, k=1):
    # сделать дома
    bought_list = np.array(bought_list)
    recommend_list = np.array(recommended_list)[:k]
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    reciprocal_rank = 1 / (relevant_indexes[0] + 1)
    return reciprocal_rank


def mean_reciprocal_rank(recommended_lists, bought_lists, k=1):
    # сделать дома
    reciprocal_rank_list = []
    for i in range(len(recommended_lists)):
        bought_list = np.array(bought_lists[i])
        recommend_list = np.array(recommended_lists[i])[:k]
        reciprocal_rank_list.append(reciprocal_rank(recommend_list, bought_list, k))
    return np.mean(reciprocal_rank_list)

In [88]:
mean_reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333

#### Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

Примеры:
    
    1. F1 score
    2. Из интернета - воплощение. Для оценки персонализации (на сколько уникальные значения выдает система для каждого пользователя)
    3. Из интернета - Expected reciprocal rank (ERR)
    4. Из интернета - PFound 